<a href="https://colab.research.google.com/github/GNuSeekK/MachineLearning/blob/main/DB_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글 드라이브 Mount

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# Frame_work , Library import

In [2]:
!pip install pymysql
 
#드라이브 마운트
import pandas as pd
import numpy as np
import random
import os, sys
import pymysql
import datetime as dt
import requests
from bs4 import BeautifulSoup
import multiprocessing as mp
from multiprocessing import Pool, Manager
from dateutil.relativedelta import relativedelta

     |████████████████████████████████| 51kB 3.0MB/s 


#Function

## 공유 메모리 관리

In [20]:
num_cores = mp.cpu_count()
manager = Manager()
db_index = manager.list(list(range(num_cores)))
progress = manager.list([0 for x in range(num_cores)])

## Function **my_pbar**
* 매개변수(parameter) : value, endvalue, bar_length(default = 20)

In [4]:
def my_pbar(value, endvalue, bar_length = 20):
    if value % 100 == 0 or value == endvalue:
        percent = float(value) / endvalue
        complete = '<' + '▮' * int(round(percent * bar_length)-1)
        remain = '▯' * (bar_length - len(complete)) + '>'
        if percent <= 0.33:
            color = 91
            sys.stdout.write(f"\r\033[{color}m Percent: [{complete+remain}] {round(percent*100,2)}%\033[0m")
        elif percent <= 0.66:
            color = 93
            sys.stdout.write(f"\r\033[{color}m Percent: [{complete+remain}] {round(percent*100,2)}%\033[0m")
        elif percent == 1:
            color = 92
            sys.stdout.write(f"\r\033[{color}m Percent: [{complete+remain}] {round(percent*100,2)}%\033[0m")
        else:
            color = 96
            sys.stdout.write(f"\r\033[{color}m Percent: [{complete+remain}] {round(percent*100,2)}%\033[0m")
        sys.stdout.flush()


## Function **db_connecting_aws**

In [5]:
def db_connecting_aws(db_name):
    path = r'/content/gdrive/My Drive/DB 정보/AWS'
    f = open(path)
    id, pw, host = f.read().split()
    db = pymysql.connect(
        user = id,
        port = 3306,
        passwd = pw,
        host = host,
        db = db_name,
        charset = 'utf8',
        cursorclass = pymysql.cursors.DictCursor
    )
    return db

### 실제 DB에 연결하기
* cpu의 갯수만큼 연결한다.
* 하나로 쓸 db, 여러개로 사용할 db_list를 따로 구성한다

In [6]:

#db 작업
 
db = db_connecting_aws('stockdb')

db_list_aws = []
num_cores = mp.cpu_count()
for _ in range(num_cores):
    db_list_aws.append(db_connecting_aws('stockdb'))

## Function insert_data
* 매개변수(parameter) : db, df, tbl
    > db를 이용하여 df(데이터프레임)을 tbl(테이블)에 replace insert 한다.

In [7]:
#데이터 넣기, 입력값 // db = 데이터베이스, df = 데이터프레임
def insert_data(db,df,tbl):
    df_list = df.values.tolist()
    slicing = 5000
    rep = int(len(df_list)/slicing) + 1 #반복횟수
    for num in range(rep):
        if num != (rep-1):
            df_sql = df_list[num*slicing:(num+1)*slicing]
        else:
            df_sql = df_list[num*slicing:]
        sql_list = []
        for value in df_sql:
            tmp = '(' + ', '.join(value) + ')'
            sql_list.append(tmp)
    
        sql = 'replace into %s values ' %tbl + ', '.join(sql_list)
        with db.cursor() as cursor:
            cursor.execute(sql)
    db.commit()

## Function ftbl_input
* 매개변수(parameter) : code
    > 종목코드가 code인 주식의 Financial Data를 ftbl에 넣는다.
    
    > insert_data를 내부적으로 사용한다.

In [21]:
def ftbl_input(code):
    tmp_index = db_index.pop(0)
    db = db_list_aws[tmp_index]
    try:
        fs_url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701' %code
        fs_page = requests.get(fs_url)
        soup = BeautifulSoup(fs_page.text, "html.parser")
        #수정평균주식수
        stock = soup.find_all('tr', class_= 'c_grid1_1 rwf acd_dep2_sub')
        CR_Data = [x.text for x in stock if '주식수' in x.text]#크롤링 데이터
        index = str(CR_Data).find('수정')
        CR_Data = str(CR_Data)[index:]
        CR_Data = CR_Data.replace('\\n', ' ')
        CR_Data = CR_Data.replace('\\xa0', '0')
        CR_Data = CR_Data.replace(',', '')
        CR_Data = CR_Data.split(' ')[:6]
        #날짜, EPS, BPS
        data_df = pd.read_html(fs_page.text, index_col = 0)[1]
        acc_month = data_df.columns.tolist()[-2].split('/')[1] #account month 결산월
        df = []
        for year in range(2018,2021):
            column = [x for x in data_df.columns.tolist() if f'{year}/{acc_month}' in x]
            if len(column) > 0:
                column = column[0]
                dateE = dt.datetime(year,int(acc_month),1) + relativedelta(months=17) + relativedelta(days=-1)
                dateE = f"'{str(dateE)[:10]}'"
                dateS = dt.datetime(year,int(acc_month),1) + relativedelta(months=5)
                dateS = f"'{str(dateS)[:10]}'"
                text_list = []
                text_list.append("'" + code + "'")
                index = [x for x in data_df.index.tolist() if 'BPS계산' in x][0]
                bps = data_df[column][index]
                text_list.append(bps)
                index = [x for x in data_df.index.tolist() if 'EPS계산' in x][0]
                eps = data_df[column][index]
                text_list.append(eps)
                if np.nan not in text_list:
                    text_list.insert(1,dateE)
                    text_list.insert(1,dateS)
                    index = data_df.columns.tolist().index(f'{year}/{acc_month}')
                    text_list.append(CR_Data[index])
                    text_list.insert(1,str(year))
                    df.append(text_list)
                else:
                    print(f'{code} None값 존재')
            else:
                print(f'{code} Column 없음')
        df = pd.DataFrame(df)
        insert_data(db,df,'ftbl')
    except:
        print(f'Error : {code}')
    finally:
        db_index.append(tmp_index)
        progress[tmp_index] = progress[tmp_index] + 1
        my_pbar(sum(progress),len(code_list))

# 데이터 Input

## 1. COMtbl 입력
* column : C_code(종목코드), C_name(회사명)
* PK : C_code

In [ ]:
df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

name_list = df['회사명'].tolist()
code_list = df['종목코드'].tolist()
code_list = list(map(str,code_list))
code_list = [x.zfill(6) for x in code_list]

code_dict = { code : value for code, value in zip(code_list,name_list)} # code와 회사 이름 딕셔너리

exist_num = pd.read_sql("select count(*) from comtbl",db).values[0][0]

# COMtbl에 회사 이름과 코드 집어넣기
sql = """
    insert ignore into comtbl values (%s, %s)
"""

with db.cursor() as cursor:
    cursor.executemany(sql,list(code_dict.items()))
    db.commit()

insert_num = pd.read_sql("select count(*) from comtbl",db).values[0][0]

print(f'작업이 완료되었습니다. {insert_num - exist_num}개의 데이터가 업데이트 되었습니다.')

#결과 확인
sql = """
    select * from comtbl
"""
result = pd.read_sql(sql,db)

작업이 완료되었습니다. 0개의 데이터가 업데이트 되었습니다.


## 2. Ftbl 입력
* column : C_code(종목코드), dateS(시작일), dateE(종료일), BPS, EPS, Issued_Shares
* Index : C_code(Foreign Key), dateS, dateE

In [22]:
code_list = pd.read_sql("select distinct C_code From comtbl",db)['C_code'].tolist()

pool = Pool(num_cores) # process 만들기
 
start_time = dt.datetime.now()
pool.map(ftbl_input, code_list)
my_pbar(100,100)
print('총 걸린시간 : %s' %(dt.datetime.now() - start_time))
pool.close()
pool.join()

 Percent: [<▮▮▮▯▯▯▯▯▯▯▯▯▯▯▯▯▯▯▯>] 20.41%036260 Column 없음
054180 Column 없음
042370 Column 없음
 Percent: [<▮▮▮▮▮▯▯▯▯▯▯▯▯▯▯▯▯▯▯>] 28.57%033600 None값 존재
033600 None값 존재
 Percent: [<▮▮▮▮▮▮▯▯▯▯▯▯▯▯▯▯▯▯▯>] 32.65%028040 Column 없음
 Percent: [<▮▮▮▮▮▮▯▯▯▯▯▯▯▯▯▯▯▯▯>] 36.73%064510 None값 존재
053110 None값 존재
 Percent: [<▮▮▮▮▮▮▮▮▮▯▯▯▯▯▯▯▯▯▯>] 48.98%088980 Column 없음
088980 Column 없음
088980 Column 없음
Error : 088980
 Percent: [<▮▮▮▮▮▮▮▮▮▮▯▯▯▯▯▯▯▯▯>] 53.06%080440 None값 존재
 Percent: [<▮▮▮▮▮▮▮▮▮▮▯▯▯▯▯▯▯▯▯>] 57.14%Error : 094800
140290 None값 존재
140290 None값 존재
140290 None값 존재
Error : 140290
 Percent: [<▮▮▮▮▮▮▮▮▮▮▮▯▯▯▯▯▯▯▯>] 61.22%086250 None값 존재
086250 None값 존재
096300 Column 없음
096300 Column 없음
096300 Column 없음
Error : 096300
087730 None값 존재
114570 None값 존재
088260 Column 없음
090740 None값 존재
 Percent: [<▮▮▮▮▮▮▮▮▮▮▮▮▮▯▯▯▯▯▯>] 69.39%212310 None값 존재
131100 Column 없음
Error : 152550
 Percent: [<▮▮▮▮▮▮▮▮▮▮▮▮▮▮▯▯▯▯▯>] 73.47%Error : 168490
 Percent: [<▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▯▯▯▯>] 77.55%329560 None값 존재
334890 Column 없음
334890 Co